In [17]:
# import libs

import numpy as np
import Laser
import gCode
import random
import Printer


# Important definition
positive angle in CCW direction

### Units
Distances: mm  
Angles: degrees

# Define functions

In [18]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [19]:
def parseCoord(gCodeObj,linestring):
    line = linestring
    X = None
    Y = None
    Z = None
    E = None
    F = None
    move = False

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]

    # check if line is ;LAYER:1 and gCodeObj.firstLayerCheck is False
    if line.startswith(';LAYER:1') and not gCodeObj.firstLayerCheck:
        gCodeObj.updateFirstLayerCheck(True)
        print("First layer check is True")

    # check if FirstLayerCheck == True and a new G1 or G0 command 
    if (gCodeObj.checkFirstLayer):
        if ((command == "G1") or (command == "G0")):
            if (gCodeObj.getHistoryMovementCommandCheck == False):  
                if (not gCodeObj.getFirstMovementCommandCheck()):
                    gCodeObj.updateFirstMovementCommandCheck(True)
            else:
                gCodeObj.updateFirstMovementCommandCheck(False)

    # check if firstLayerCheck is True
    if gCodeObj.firstLayerCheck:
        # if command == G1 or G0 than move = True
        if command == 'G1' or command == 'G0':
            move = True
        if command == 'G1':
            gCodeObj.updateNumberOfG1()
            
        # check if line contains ; which means it is a comment or one of the instructions lines in the beginning and needs to be skipped 
        if not ';' in line:
            # parse the X Y Z coords from string to float, and the values of F and E
            if 'X' in line:
                X = float(line.split('X')[-1].split(' ')[0])
            elif 'X' not in line:
                X = None
            if 'Y' in line:
                Y = float(line.split('Y')[-1].split(' ')[0])
            elif 'Y' not in line:
                Y = None
            if 'Z' in line:
                Z = float(line.split('Z')[-1].split(' ')[0])
            elif 'Z' not in line:
                Z = None
            if 'E' in line:
                E = float(line.split('E')[-1].split(' ')[0])
            elif 'E' not in line:
                E = None
            if 'F' in line:
                F = float(line.split('F')[-1].split(' ')[0])
            elif 'F' not in line:
                F = None
    else:
        X = None
        Y = None
        Z = None
        E = None
        F = None
    return move, X, Y, Z, E, F, command, linestring


In [20]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    try:
        return np.sqrt((x1-x0)**2 + (y1-y0)**2)
    except:
        print(x0, y0, x1, y1)


In [21]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = np.arctan2((y1-y0),(x1-x0))
   
    # convert to degrees
    newAngle = np.rad2deg(newAngle)
    
    # normalize angle
    #newAngle = np.mod(newAngle, 360)
    
    return newAngle

In [22]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getNormAngles()
    
    # find the closest angle in the list of angles
    closestAngleIndx = np.argmin(np.abs(currentAngles - targetAngle))
    closestAngle = currentAngles[closestAngleIndx]

    # calculte delta bewteen target and closest angle
    deltaAngle = targetAngle - closestAngle

    # update the angle of the laser
    #LaserObj.updateAngles(deltaAngle)

    return closestAngleIndx, deltaAngle

    

In [23]:

def convertAngleToSteps(angle):
    return angle


In [24]:
def deactivateAllLaser(newFilePath, laserObj, laserStringOffset):
    for l in range(len(laserObj.LaserStates)):
        newLine02 = "M106 P" + str(l+laserStringOffset) + " S0.0\n"
        laserObj.updateLaserStates(l, 0)
        with open(newFilePath, 'a') as f:
            f.write(newLine02)
            f.close()  
    return

In [25]:
def deativateONLaser(newFilePath, laserObj, laserStringOffset):
    for l in range(len(laserObj.LaserStates)):
        if(laserObj.LaserStates[l] == 1):
                # deactivate
                laserObj.updateLaserStates(l, 0)
                deactCmd = "M106 P" + str(l+laserStringOffset) + " S0\n"
                with open(newFilePath, 'a') as f:
                    f.write(deactCmd)
                    f.close() 
                

In [26]:
def appendToFile(newFilePath, string):
    with open(newFilePath, 'a') as f:
        f.write(string)
        f.close()
    return

In [27]:
def processGcode(gCodeFile, thresholdDistance, thresholdAngle, laserObj, newFilePath, gCodeObj):

    ### processing parameters
    #  define string offset for physical laser connection
    laserStringOffset = 2

    # speeds for G0 and G1
    F0 = 2700
    F1 = 1800

    ### Include start file parameter
    # deactivate all lasers
    deactivateAllLaser(newFilePath=newFilePath, laserObj=laserObj, laserStringOffset=laserStringOffset)

    # add movement to X0.0 Y0.0 Z0.2 for test line bottom left corner
    moveCMD = "G1 X0 Y0 Z0.2"

    # activate fan
    fanPWM = 0.2
    activateFanCMD = "M106 P6 S" + str(fanPWM) + "\n"
    appendToFile(newFilePath, activateFanCMD)

    # laser PWM
    laserPWM = 0.8


    

    ### start loop through entire gcode
    for i in range(gCodeFile.shape[0]):

        # read line and compare to current pos which is saved in gCodeObj
        move1, X1, Y1, Z1, E1, F1, command1, line1 = parseCoord(gCodeObj, gCodeFile[i])
        print(command1)

        # check for firstMovementCommandCheck and write coords into currentPos, no further processing
        if(gCodeObj.getFirstMovementCommandCheck):
            gCodeObj.updateCurrentPos([X1, Y1, Z1, 0]) 
        
        # get current pos
        X0, Y0, Z0, A = gCodeObj.getCurrentPos()

        # calculate distance
        distance = calcDistance(x0=X0, y0=Y0, x1=X1, y1=Y1)

        # update total distance 
        if(command1 == "G1"):
            gCodeObj.updateTotalLengthOfG1(distance)


        # only execute rotation and laser ON OFF if line is a G1
        if(command1 == "G1" or command1 == "G0"):

            # calculate angle
            angle = calcAngle(x0=X0, y0=Y0, x1=X1, y1=Y1)
            lastAngle = gCodeObj.getLastAngle()

            # update last angle in gCodeObj
            gCodeObj.updateLastAngle(angle)

            # normalize the target angle between 0 and 360 degrees
            targetAngle = np.mod(angle, 360)

            # find the closest angle to the target angle
            closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, laserObj)

            # get absolute angle for rotation command
            absAngle = laserObj.getAbsLaserPos()
            absAngle += deltaAngle

            # update currentPos in gCodeObj
            gCodeObj.updateCurrentPos([X1, Y1, Z1, angle])

            # command string will be filled during the if statements
            command = ""




            # check if any laser is activated, if yes and not the closestIndx than deactivate
            for l in range(len(laserObj.LaserStates)):
                if(laserObj.LaserStates[l] == 1):
                    if(laserObj.getLaserStates(l) != closestAngleIndx):
                        # deactivate
                        deactCmd = "M106 P" + str(l+laserStringOffset) + " S0\n"
                        appendToFile(newFilePath=newFilePath, string=deactCmd)
                        laserObj.updateLaserStates(l, 0)
                
            if(command1 == "G1"):
                if(1):
                ### MODE Straights   
                    
                    # rotate laser
                    command += "G0 F" + str(F0) + " A" + str(absAngle) + '\n'
                    

                    # if laser not turned on yet, activate
                    if(laserObj.getLaserStates(closestAngleIndx) == 0):
                        # activate laser 
                        command += "M106 P" + str(closestAngleIndx+laserStringOffset) + " S" + str(laserPWM) + "\n"
                        # update laserstate
                        laserObj.updateLaserStates(closestAngleIndx, 1)

                    command += ";Laserstates " + str(laserObj.LaserStates) + "\n"
                    # insert G1 line
                    command += line1
                    # deactivate laser
                    # command += "M106 P" + str(closestAngleIndx+laserStringOffset) + " S" + str(0.0) + "\n"
                    


                elif(0):
                ### MODE Curves
                    f = 1

                    # check if the needed laser is 
            else:
                command = line1
            # write to file
            appendToFile(newFilePath=newFilePath, string=command)


            # update absolute position of laser and absolut movement
            laserObj.updateAngles(deltaAngle)

            # update states of the lasers
            laserObj.updateLaserStates(closestAngleIndx, 1)



    
        else:
            # check if any laser is on and turn off
            deativateONLaser(newFilePath, laserObj, laserStringOffset)
            # append line0 to newFilePath
            with open(newFilePath, 'a') as f:
                f.write(line1)
                f.close()


In [28]:
def postProcessGcode(gCodeArray, thresholdDist, thresholdAngle, LaserObj, newFilePath, gCodeObj):

    # define string offset for physical laser connection
    laserStringOffset = 2
    
    # turn off all lasers
    for l in range(len(LaserObj.LaserStates)):
        newLine02 = "M106 P" + str(l+laserStringOffset) + " S0.0\n"
        LaserObj.updateLaserStates(l, False)
        with open(newFilePath, 'a') as f:
            f.write(newLine02)
            f.close()    


    for i in range(gCodeArray.shape[0]-1):
        move0, X0, Y0, Z0, command0, line0 = parseCoord(gCodeObj, gCodeArray[i])
        move1, X1, Y1, Z1, command1, line1 = parseCoord(gCodeObj,gCodeArray[i+1])
        try:
            # update lastPos if move1 is False and move0 is True
            if move0 and not move1:
                gCodeObj.updateCurrentPos([X0, Y0, Z0])

            # PWM power
            pwm = 1

            # clear Extruder for testing
            if(0):
                # copy line0 until reaches E 
                line0 = line0.split('E')[0] + '\n'

            # if move0 is not true, get the last position from the gCodeObj
            if not move0:
                X0 = gCodeObj.getCurrentPos()[0]
                Y0 = gCodeObj.getCurrentPos()[1]
                Z0 = gCodeObj.getCurrentPos()[2]


            # check if both lines are moves
            #if move0 and move1:
            if move1:
                # update the current position
                gCodeObj.updateCurrentPos([X1, Y1, Z1])


                # check if the distance between the two points is larger than the threshold
                dist = calcDistance(X0, Y0, X1, Y1)
                gCodeObj.updateTotalLengthOfG1(dist)

                # calculate the angle between the two points
                targetAngle = calcAngle(X0, Y0, X1, Y1)
                lastAngle = gCodeObj.getLastAngle()
                
                # update the last angle
                gCodeObj.updateLastAngle(targetAngle)

                # noralize the traget angle
                targetAngle = np.mod(targetAngle, 360)

                # find the closest angle to the target angle
                closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)
                # update the angle of the laser
                LaserObj.updateAngles(deltaAngle)
                # convert the angle to steps
                steps = convertAngleToSteps(deltaAngle)
                    

                # MODUS LINEARE 
                if dist >= thresholdDist:            # and not (abs(lastAngle - targetAngle)>45)
                    # activate laser PWM signal
                    newLine02 = "\nM106 P" + str(closestAngleIndx+laserStringOffset) + " S" + str(pwm) +'\n'
                    absAngle = LaserObj.getAbsLaserPos()
                    newLine02 += "G1 F10000 A" + str(absAngle) + '\n'
                    # insert line1 and delete '\n'
                    nline0 = line1

                    # remove '\n' from nline0
                    nline0 = nline0[:-1]

                    # add comment
                    nline0 += "; MODUS LINEAR\n"
                    
                    # update updateLastActivatedLaser
                    LaserObj.updateLastActivatedLaser(closestAngleIndx)
                
                    # deactivate laser PWM signal
                    newLine04 = "M106 P" + str(closestAngleIndx+laserStringOffset) + " S0.0\n"

                    # update ON OFF State
                    LaserObj.updateLaserStates(closestAngleIndx, False)

                    # update last angle
                    gCodeObj.updateLastAngle(absAngle)

                    # append new lines to newFilePath
                    with open(newFilePath, 'a') as f:
                        f.write(newLine02)
                        f.write(nline0)
                        f.write(newLine04)
                        f.close()

                # MODUS KURVE
                elif (dist < thresholdDist and (abs(LaserObj.lastAngle-deltaAngle)<thresholdAngle)):

                    # drive the motion and turn the angle
                    absAngle = LaserObj.getAbsLaserPos()
                    # insert line0 and delete '\n'
                    nline0 = line1
                    # remove '\n' from nline0
                    nline0 = nline0[:-1]      

                    # check if next laser == last laser
                    if (closestAngleIndx != LaserObj.getLastActivatedLaser() or LaserObj.LaserStates[closestAngleIndx] == False) :
                        # deactivate all lasers that are active
                        for l in range(len(LaserObj.LaserStates)):
                            if LaserObj.LaserStates[l]:
                                newLine02 = "M106 P" + str(l+laserStringOffset) + " S0.0\n"
                                with open(newFilePath, 'a') as f:
                                    f.write(newLine02)
                                    f.close()

                        if command1 == 'G1':                     
                            # activate laser PWM signal
                            newLine02 = "M106 P" + str(closestAngleIndx+laserStringOffset) + " S" + str(pwm) +'\n'
                            newLine03 = nline0 + " A" + str(absAngle) + ' ;MODUS KURVE\n'
                        else:
                            newLine03 = nline0 + ' ;MODUSE KURVE OHNE Laser\n'
                        
                        
                        
                        # update last activated laser
                        LaserObj.updateLastActivatedLaser(closestAngleIndx)
                        # update ON OFF state
                        LaserObj.updateLaserStates(closestAngleIndx, True)

                        with open(newFilePath, 'a') as f:
                                    f.write(newLine02)
                                    f.close()
                    
                    
                    # update last angle
                    gCodeObj.updateLastAngle(absAngle)

                    # append new lines to newFilePath
                    with open(newFilePath, 'a') as f:
                        f.write(newLine03)
                        f.close()

            

                # else append line
                # append line0 to newFilePath
                
                else:
                    with open(newFilePath, 'a') as f:
                        f.write(line1)
                        f.close()

            else:
                # append line0 to newFilePath
                with open(newFilePath, 'a') as f:
                    f.write(line1)
                    f.close()

        except:
            print("Error in line: ", i)
            print(line0)
            print(line1)

    # append last lines
    # turn off all lasers
    for l in range(len(LaserObj.LaserStates)):
        if LaserObj.LaserStates[l]:
            print("turn off laser: ", l)
            print(LaserObj.LaserStates)
            newLine02 = "M106 P" + str(l+laserStringOffset) + " S0.0\n"
            with open(newFilePath, 'a') as f:
                f.write(newLine02)
                f.close()
            
        

    return True

In [29]:
# Add start lines to GCODE


# Test functions

In [30]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,0,0))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [31]:
# test the parseCoord function
if(0):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        moveBool, x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(0):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

# Main 

In [32]:
# initiate Printer object, for duration calculation of process
Prusa = Printer.Printer(velocity_trans=100, velocity_angular=100)

# read the gcode file
gcode = read_gcode_file("Input/PI3MK3M_Square_50mm.gcode")
gCodeObj = gCode.gCode("gCode")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

# threshold for the distance between two points 
minDistance = 2
# threshold for the angle between two points
minAngle = 5

# check index in Output/index.txt
with open("Output/index.txt", 'r') as f:
    index = int(f.read())
    f.close()

# create new file
newFilePath = "Output/GCODE_postprocessed_" + str(index) + ".gcode"

# iterate index in Output/index.txt
with open("Output/index.txt", 'w') as f:
    f.write(str(index+1))
    f.close()

# post process the gcode
print(processGcode(gcode, minDistance, minAngle, laser, newFilePath, gCodeObj))


(120,)
First layer check is True
;LAYER:1

None None None None
M106
None None None None
;TYPE:WALL-INNER

None None None None
;MESH:Square_50mm.stl

None None None None
G1
G1
G1
G1
G0
;TYPE:WALL-OUTER

None None None None
G1
G1
G1
G1
G0
G0
;TYPE:SKIN

None None None None
G1
G1
G1
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0


None None None None
G0
G0
G0
;TIME_ELAPSED:456.519934

None None None None
;LAYER:2

None None None None
;TYPE:WALL-INNER

None None None None
;MESH:Square_50mm.stl

None None None None
G1
G1
G1
G1
G0
;TYPE:WALL-OUTER

None None None None
G1
G1
G1
G1
G0
G0
;TYPE:SKIN

None None None None
G1
G1
G1
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
G1
G0
None


In [33]:
print(gCodeObj.firstLayerCheck)
print(laser.absLaserMovement)
print("Total lenght of G1 commands:\t " + str(gCodeObj.getTotalLengthOfG1()) + " mm")
print("Total angle turned: \t" + str(laser.getAbsLaserMovement()))
print("Total number of G1 commands: \t\t" + str(gCodeObj.getNumberOfG1()))



True
0.0
Total lenght of G1 commands:	 0.0 mm
Total angle turned: 	0.0
Total number of G1 commands: 		60


In [34]:
print(laser.LaserStates)

[1, 0, 0, 0]
